In [1]:
from window_generator import WindowGenerator
import tensorflow as tf
import numpy as np
import pandas as pd
# from read_data import *
from process_data import *
from methods import *
MAX_EPOCHS = 20
VOICE = 0

In [2]:
w1 = WindowGenerator(input_width=6, label_width=1, shift=1,batch_size=2, voice_number=0)
print(w1)

for example_inputs, example_labels in w1.train.take(1):
    print(f"Inputs shape (batch, time, features): {example_inputs.shape}")
    print(f"Labels shape (batch, time, features): {example_labels.shape}")
    
    print(example_inputs)
    print(example_labels)

tf.squeeze(example_labels, axis=1)

# # t = [0] * 20
# # t[0] = 1
# # t = np.array(t)
# # print(t)
# # print(w1.shift_conversion_key)
# # print(np.where(t == max(t)))


# # get_shift_from_probability(t, w1.shift_conversion_key)


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/Users/Daan/Documents/Study/MSc/Machine Learning/project/Completing-Bach-ML/window_generator.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.train_df.iloc[:,:5] = (self.train_df.iloc[:,:5] - self.mean_train) / self.std_train
/Users/Daan/Documents/Study/MSc/Machine Learning/project/Completing-Bach-ML/window_generator.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See

Total window size: 7
Input indices: [0 1 2 3 4 5]
Input columns name(s): ['log_pitch', 'chroma_x', 'chroma_y', 'c5_x', 'c5_y', 'shift_0', 'shift_1', 'shift_2', 'shift_3', 'shift_4', 'shift_5', 'shift_6', 'shift_7', 'shift_8', 'shift_9', 'shift_10', 'shift_11', 'shift_12', 'shift_13', 'shift_14', 'shift_15', 'shift_16', 'shift_17', 'shift_18', 'shift_19']Label indices: [6]
Label column name(s): ['shift_0', 'shift_1', 'shift_2', 'shift_3', 'shift_4', 'shift_5', 'shift_6', 'shift_7', 'shift_8', 'shift_9', 'shift_10', 'shift_11', 'shift_12', 'shift_13', 'shift_14', 'shift_15', 'shift_16', 'shift_17', 'shift_18', 'shift_19']
Inputs shape (batch, time, features): (2, 6, 25)
Labels shape (batch, time, features): (2, 1, 20)
tf.Tensor(
[[[ 0.27656335  1.0811402   0.5515396  -0.89000624 -0.28040573
    1.          0.          0.          0.          0.
    0.          0.          0.          0.          0.
    0.          0.          0.          0.          0.
    0.          0.          0.     

<tf.Tensor: shape=(2, 20), dtype=float32, numpy=
array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.]], dtype=float32)>

In [3]:
def compile_and_fit(model, window, patience=3, verbose=0):
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                      patience=patience,
                                                      mode='min',
                                                     restore_best_weights=True)
#  Early stopping when there is no improvement in loss. 
    model.compile(loss=tf.losses.CategoricalCrossentropy(from_logits=True),
                  optimizer=tf.optimizers.Adam(),
                  metrics=[tf.metrics.CategoricalAccuracy(), tf.metrics.CategoricalCrossentropy(from_logits=True),tf.metrics.MeanAbsoluteError()])

# At the end of each epoch, the model will iterate over the validation dataset and compute the validation loss and validation metrics.
    history = model.fit(window.train, epochs=MAX_EPOCHS,
                        validation_data=window.val,
                        callbacks=[early_stopping],
                        verbose=verbose)
    return history

In [4]:
multi_step_window = WindowGenerator(input_width=54, label_width=1, shift=1, batch_size=1, voice_number=VOICE)
count = 0
for example_inputs, example_labels in multi_step_window.train:
    count+= 1
    tf.squeeze(example_labels, axis=1)
print(count)


for example_inputs, example_labels in multi_step_window.train.take(1):
    print(f"Inputs shape (batch, time, features): {example_inputs.shape}")
    print(f"Labels shape (batch, time, features): {example_labels.shape}")

print(multi_step_window.train)
# print(len(multi_step_window.label_columns_indices))

2045
Inputs shape (batch, time, features): (1, 54, 25)
Labels shape (batch, time, features): (1, 1, 20)
<MapDataset shapes: ((None, 54, 25), (None, 1, 20)), types: (tf.float32, tf.float32)>


In [ ]:
n_output_nodes = len(multi_step_window.label_columns_indices)

# initializer = tf.keras.initializers.RandomUniform(minval=-0.50, maxval=0.50)
linear_multi_step = tf.keras.Sequential([
    # Shape: (time, features) => (time*features)
    tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(units=64, kernel_initializer=initializer, activation='relu'),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=n_output_nodes),
    # Add back the time dimension.
    # Shape: (outputs) => (1, outputs)
    tf.keras.layers.Reshape([1, -1]),
])
print('Input shape:', multi_step_window.example[0].shape)
print('Output shape:', linear_multi_step(multi_step_window.example[0]).shape)

# Train the model
history = compile_and_fit(linear_multi_step, multi_step_window, verbose=2)

print('Note that these are the LAST model metrics and not the BEST model metrics because of early stopping with restore_best_weights ')
print(
    "epochs: {}, loss: {:0.4f}, mean_absolute_error: {:0.4f}, val_loss: {:0.4f}, val_mean_absolute_error: {:0.4f}".format(
        len(history.history['loss']),
        history.history["loss"][-1],
        history.history["mean_absolute_error"][-1],
        history.history["val_loss"][-1],
        history.history["val_mean_absolute_error"][-1],
    )
)

# Evaluate the model
print("Evaluate")
result = linear_multi_step.evaluate(multi_step_window.test)
dict(zip(linear_multi_step.metrics_names, result))

In [ ]:
# n_output_nodes = len(multi_step_window.label_columns_indices)

# rnn = tf.keras.Sequential([
#     tf.keras.layers.LSTM(64),
#     tf.keras.layers.Dense(units=n_output_nodes, activation='softmax'),
#     # Add back the time dimension.
#     # Shape: (outputs) => (1, outputs)
#     tf.keras.layers.Reshape([1, -1]),
# ])
# print('Input shape:', multi_step_window.example[0].shape)
# print('Output shape:', rnn(multi_step_window.example[0]).shape)

# # Train the model
# history = compile_and_fit(rnn, multi_step_window, verbose=1)

# print('Note that these are the LAST model metrics and not the BEST model metrics because of early stopping with restore_best_weights ')
# print(
#     "epochs: {}, loss: {:0.4f}, mean_absolute_error: {:0.4f}, val_loss: {:0.4f}, val_mean_absolute_error: {:0.4f}".format(
#         len(history.history['loss']),
#         history.history["loss"][-1],
#         history.history["mean_absolute_error"][-1],
#         history.history["val_loss"][-1],
#         history.history["val_mean_absolute_error"][-1],
#     )
# )

# # Evaluate the model
# print("Evaluate")
# result = rnn.evaluate(multi_step_window.test)
# dict(zip(rnn.metrics_names, result))

In [ ]:
# Get last window
def predict_new_pitches(model, n=100):
    last_window = None
    last_pitch = None
    all_predicted_pitches = []
    all_probs = []

    for i in range(n):
        if last_window is None:
            # Use an iterator to get the last window of the dataset. 
            for last_window, labels in multi_step_window.test_no_shuffle.as_numpy_iterator():
                pass

        # Add dimension to input
        probabilities = model.predict(np.array([last_window]))[0]
        probabilities = probabilities[0]
        all_probs.append(probabilities)
        
        predicted_shift = get_shift_from_probability(
            probabilities,
            key=multi_step_window.shift_conversion_key,
            method=SelectionMethod.PROB, n=10,
        )
        
        # Get latest pitch of the dataset
        if last_pitch is None:
            last_pitch = get_voice(VOICE)[-1]
            
        predicted_pitch = last_pitch + predicted_shift
        all_predicted_pitches.append(predicted_pitch)
        
        pitch_features = get_pitch_features(predicted_pitch)
        shift_one_hot = [0] * len(multi_step_window.shift_conversion_key)
        shift_one_hot[multi_step_window.shift_conversion_key.index(predicted_shift)] = 1

        last_pitch = predicted_pitch

        # create new dataframe
        new_df = pd.DataFrame(dtype=float, index=[0])
        (
            new_df["log_pitch"],
            new_df["chroma_x"],
            new_df["chroma_y"],
            new_df["c5_x"],
            new_df["c5_y"],
        ) = pitch_features
        
        # - Normalise features
        new_df = (new_df - multi_step_window.mean_train) / multi_step_window.std_train
        features = new_df.values.tolist()[0] + shift_one_hot
        # - Make new window based on previous timesteps with a slide of 1 and the normalised features
        last_window = np.append(last_window[1:], [features], axis=0)
    return all_predicted_pitches, all_probs

In [ ]:
print(predict_new_pitches(linear_multi_step, n=1))



# probability_model = tf.keras.Sequential([linear_multi_step, tf.keras.layers.Softmax(axis=-1)])

# predictions, probs = predict_new_pitches(probability_model, n=1)
# probability_model.summary()

# print(probs[-1])
# print(sum(probs[-1]))
# print(predictions)
# # print(probs[-1])
# # sum(probs[-1])
# # # wg = WindowGenerator(input_width=6, label_width=1, shift=1, batch_size=1)
# # print(f'train {sum(1 for _ in wg.train)} val {sum(1 for _ in wg.val)} test {sum(1 for _ in wg.test)} ')
# # print(f'full dataset {sum(1 for _ in wg.full_dataset)}')
# # print(f'data size {wg.df.shape[0]} | file voice size: {len(get_voice(0))}')

# # max(get_voice(2))

